In [ ]:
MODEL_NAME = "tft-nb-4"

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from multiprocessing.dummy import freeze_support
import os
import sys
from dotenv import load_dotenv
load_dotenv()
os.environ['WANDB_NOTEBOOK_NAME'] = 'pytorch_stats_own_data.ipynb'
os.environ['WANDB_API_KEY'] = os.getenv('WANDB_API_KEY')

nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch

from darts import TimeSeries
from darts.models import TFTModel
from darts.dataprocessing.transformers import Scaler, MissingValuesFiller
from darts.utils.likelihood_models import QuantileRegression
from darts.metrics import mape, r2_score, rmse, mse
from darts import TimeSeries

import helper
import glob

import wandb
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.strategies import DDPStrategy, ddp2
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

from tqdm.contrib.concurrent import process_map
import tqdm


AVAILABLE_GPUS = torch.cuda.device_count()
AVAILABLE_CPUS = os.cpu_count()
TRAINING_DATA_PATH = "../../../Data/london_clean/*.csv"

print(f"Available GPUs: {AVAILABLE_GPUS}")
print(f"Available CPUs: {AVAILABLE_CPUS}")


# Data

Taking the first 300 households from the London Dataset and converting them to a Darts TimeSeries.

In [ ]:
# my_time_series_dataset = []
# my_cov_series = []
# for i in tqdm.tqdm(sorted(glob.glob("../../../Data/london_clean/*.csv"))[:100]):
#     df = pd.read_csv("../../../Data/London_weather_2011-2014.csv")
#     df["DateTime"] = pd.to_datetime(df["DateTime"])
#     df2 = pd.read_csv(i)
#     df2["DateTime"] = pd.to_datetime(df2["DateTime"])
#     df = df.merge(df2, on="DateTime", how="right")
#     df.fillna(method="ffill" ,inplace=True)
#     series = TimeSeries.from_dataframe(df, time_col="DateTime", value_cols=["KWHhh"], freq="30min", fill_missing_dates=True, fillna_value=True).astype(np.float32)
#     cov_series =  TimeSeries.from_dataframe(df, time_col="DateTime", value_cols=["Temperature_C", "Humidity_%", "Dew_Point_C"], freq="30min", fill_missing_dates=True, fillna_value=True).astype(np.float32)
#     my_time_series_dataset.append(series)
#     my_cov_series.append(cov_series)

In [ ]:
import time
weather = pd.read_csv("../../../Data/London_weather_2011-2014.csv")
weather["DateTime"] = pd.to_datetime(weather["DateTime"])

def reader(x):
    df2 = pd.read_csv(x)
    df2["DateTime"] = pd.to_datetime(df2["DateTime"])
    df = weather.merge(df2, on="DateTime", how="right")
    df.fillna(method="ffill" ,inplace=True)
    series = TimeSeries.from_dataframe(df, time_col="DateTime", value_cols=["KWHhh"], freq="30min", fill_missing_dates=True, fillna_value=True).astype(np.float32)
    covarient = TimeSeries.from_dataframe(df, time_col="DateTime", value_cols=["Temperature_C", "Humidity_%", "Dew_Point_C"], freq="30min", fill_missing_dates=True, fillna_value=True).astype(np.float32)
    list = [series, covarient]
    return list


def splitter():
    file_list = sorted(glob.glob("../../../Data/london_clean/*.csv"))[:1000]
    if file_list == []:
        raise Exception("No files found")
    return process_map(reader, file_list, chunksize=20)

if __name__ == "__main__":
    freeze_support()
    my_time_series_dataset = splitter()

    values_series = [x[0] for x in my_time_series_dataset]
    covariance_series = [x[1] for x in my_time_series_dataset]

In [ ]:
values_series[19].plot()
covariance_series[19].plot()

In [ ]:
## sets
training_sets = []
validation_sets = []
for x in values_series:
    train, val = x.split_after(0.85)
    training_sets.append(train)
    validation_sets.append(val)

In [ ]:
##normalize the data
scaler = Scaler()

train_transformed = []
val_transformed = []
for x in training_sets:
    train_transformed.append(scaler.fit_transform(x))
for x in validation_sets:
    val_transformed.append(scaler.fit_transform(x))


## covariate series
cov_transformed = []
for x in covariance_series:
    cov_transformed.append(scaler.fit_transform(x))

In [ ]:
train_transformed[19].plot()
val_transformed[19].plot()
cov_transformed[19].plot()

# Model

We create Temporal Fusion Transformer model that utilizes the GPU, Weights, Biases logger and early stopping callback.

## Early stopping

An early stopping callback is used to stop the training if the validation loss does not improve after a certain number of epochs.


In [ ]:
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
early_stop_callback = EarlyStopping(
    monitor="val_loss",
    min_delta=0.1,
    patience=10,
    verbose=True,
    mode="min"
    )

In [ ]:
lowest_q, low_q, high_q, highest_q = 0.01, 0.1, 0.9, 0.99
label_q_outer = f"{int(lowest_q * 100)}-{int(highest_q * 100)}th percentiles"
label_q_inner = f"{int(low_q * 100)}-{int(high_q * 100)}th percentiles"

In [ ]:
encoders = {
    # "datetime_attribute": {"future": ["DateTime"], "past": ["DateTime"]},
    "position": {"past": ["absolute"], "future": ["relative"]},
    "transformer": Scaler(),
}

In [ ]:
wandb_logger = WandbLogger(project="Digital-Energy", log_model=True)


# input chunk = The length of the input sequence fed to the model
# output chunk = The length of the output sequence predicted by the model


# default quantiles for QuantileRegression
quantiles = [
    0.01,
    0.05,
    0.1,
    0.15,
    0.2,
    0.25,
    0.3,
    0.4,
    0.5,
    0.6,
    0.7,
    0.75,
    0.8,
    0.85,
    0.9,
    0.95,
    0.99,
]

input_chunk_length = 96
forecast_horizon = 96

model = TFTModel(
    input_chunk_length=input_chunk_length,
    output_chunk_length=forecast_horizon,
    hidden_size=64,
    lstm_layers=2,
    num_attention_heads=4,
    dropout=0.04,
    batch_size=512,
    n_epochs=20,
    add_relative_index=True,
    add_encoders=encoders,
    work_dir="../../../Models",
    save_checkpoints=False,
    pl_trainer_kwargs={
    "enable_progress_bar": True,
    "enable_model_summary": True,
    "accelerator": "gpu",
    "devices": [1],
    "logger": wandb_logger,
    "callbacks": [early_stop_callback]
    },
    # likelihood=QuantileRegression(
    #     quantiles=quantiles
    # ),  # QuantileRegression is set per default
    random_state=42,
)

In [ ]:
#wandb_logger.watch(model_nbeats) # sadly this feature does not work for Darts models
model.fit(series=train_transformed, val_series=val_transformed,  num_loader_workers=AVAILABLE_CPUS, future_covariates=cov_transformed, val_future_covariates=cov_transformed)

In [ ]:
START = 3000
weather = pd.read_csv("../../../Data/London_weather_2011-2014.csv")
weather["DateTime"] = pd.to_datetime(weather["DateTime"]) 
for i, x in enumerate(sorted(glob.glob("../../../Data/london_clean/*.csv"))[START:START+1]):
    df = pd.read_csv(x)
    df["DateTime"] = pd.to_datetime(df["DateTime"])
    df = weather.merge(df, on="DateTime", how="right")
    df.fillna(method="ffill" ,inplace=True)
    series = TimeSeries.from_dataframe(df, time_col="DateTime", value_cols=["KWHhh"], freq="30min", fill_missing_dates=True, fillna_value=True).astype(np.float32)
    covarient = TimeSeries.from_dataframe(df, time_col="DateTime", value_cols=["Temperature_C", "Humidity_%", "Dew_Point_C"], freq="30min", fill_missing_dates=True, fillna_value=True).astype(np.float32)
    series = series[5000:5250]


    pred_series = model.historical_forecasts(
        series,
        future_covariates=covarient,
        forecast_horizon=1,
        stride=1,
        retrain=False,
        verbose=True,
    )

    print(f"rmse: {rmse(series, pred_series)}.")
    print(f"R2 score: {r2_score(series, pred_series)}.")

    # helper.display_forecast(pred_series, series, "1 day", save=True, fig_name=f"{x.split('/')[-1]}-test", fig_size=(20,10))
    plt.figure(figsize=(20, 10))
    pred_series.plot(low_quantile=lowest_q, high_quantile=highest_q, label=label_q_outer)
    series.plot()


In [ ]:
series.plot()

In [ ]:
preds = model.predict(1, series, future_covariates=covarient, num_samples=50)
series.plot()
preds.plot()
plt.legend()
# save the plot
plt.savefig("../../../Plots/TFT_weather5.png")

In [ ]:
preds

In [ ]:
plt.figure(figsize=(20, 10))
series.plot()
pred_series.plot(low_quantile=lowest_q, high_quantile=highest_q, label=label_q_outer)
plt.legend()
# save the plot
plt.savefig("../../../Plots/TFT_weather3.png")


In [ ]:
print(f"rmse: {rmse(series, pred_series)}.")
print(f"R2 score: {r2_score(series, pred_series)}.")

fig = helper.display_forecast(pred_series, series, "1 day", save=True, fig_name=f"{i}", model_name="test", fig_size=(20,10))

wandb.log({
        "mape": mape(series, pred_series),
        "mse": mse(series, pred_series),
        "rmse": rmse(series, pred_series),
        "r2": r2_score(series, pred_series),
        "result": fig
})

In [ ]:
START = 3000
SAMPLES = 5

weather = pd.read_csv("../../../Data/London_weather_2011-2014.csv")
weather["DateTime"] = pd.to_datetime(weather["DateTime"])
scaler = Scaler()

lowest_q, low_q, high_q, highest_q = 0.01, 0.1, 0.9, 0.99
label_q_outer = f"{int(lowest_q * 100)}-{int(highest_q * 100)}th percentiles"
label_q_inner = f"{int(low_q * 100)}-{int(high_q * 100)}th percentiles"

for i, x in enumerate(sorted(glob.glob(TRAINING_DATA_PATH))[START:START+SAMPLES]):

    df = pd.read_csv(x)
    df["DateTime"] = pd.to_datetime(df["DateTime"])
    df = weather.merge(df, on="DateTime", how="right")
    df.fillna(method="ffill" ,inplace=True)
    series = TimeSeries.from_dataframe(df, time_col="DateTime", value_cols=["KWHhh"], freq="30min", fill_missing_dates=True, fillna_value=True).astype(np.float32)
    covarient = TimeSeries.from_dataframe(df, time_col="DateTime", value_cols=["Temperature_C", "Humidity_%", "Dew_Point_C"], freq="30min", fill_missing_dates=True, fillna_value=True).astype(np.float32)
    MID = len(series)//2
    series = series[MID:MID+600]
    series = scaler.fit_transform(series)


    pred_series = model.historical_forecasts(
        series,
        future_covariates=covarient,
        forecast_horizon=96,
        stride=1,
        retrain=False,
        verbose=True,
    )

    print(f"rmse: {rmse(series, pred_series)}.")
    print(f"R2 score: {r2_score(series, pred_series)}.")

    # fig = helper.display_forecast(pred_series, series, "1 day", save=False, fig_name=f"{x.split('/')[-1]}-test", fig_size=(20,10))
    plt.figure(figsize=(20, 10))
    plt.ylim(0, 3)
    plt.gca().set_alpha(1)
    series.plot()
    plt.title(f"{MODEL_NAME} - {x.split('/')[-1]} - MSE: {mse(series, pred_series)}")
    pred_series.plot(low_quantile=lowest_q, high_quantile=highest_q, label=label_q_outer)
    fig = plt
    # wandb.log({
    #         "mape": mape(series, pred_series),
    #         "mse": mse(series, pred_series),
    #         "rmse": rmse(series, pred_series),
    #         "r2": r2_score(series, pred_series),
    #         "result": fig
    # })
    name = x.split('/')[-1].split('.')[0]
    if not os.path.exists(f"../../../Plots/{MODEL_NAME}/"):
        os.makedirs(f"../../../Plots/{MODEL_NAME}/")
    plt.savefig(f"../../../Plots/{MODEL_NAME}/{name}.png")

# Loading checkpoints of the model

loading the best checkpoint of the model. To compare the results of the model with the previous one.

In [ ]:
MODEL_NAME = "tft-2000-3"

In [ ]:
# load the model
model = TFTModel.load_from_checkpoint(work_dir="../../../Models/", model_name=MODEL_NAME, best=True)

In [ ]:
model.trainer_params.update({"accelerator": "cpu"})

In [ ]:
helper.eval(model, future_covariates=True, data_normalized=True)